<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza
!pip install conllu
!git clone https://github.com/IKeeso/Project9_data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 27.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

In [2]:
import stanza
from stanza.utils.conll import CoNLL
from conllu import parse
from conllu import TokenList
import re

!ls Project9_data


expressen.txt  Project.ipynb


In [3]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
def tag_and_convert(filename):
  %cd /content/Project9_data
  try:
    with open(filename, 'r', encoding='utf-8') as file:
      text = file.read()
      file.close()
  except:
    text = filename
  doc = nlp(text)
  pathname = filename.split(".")[0]
  conllufilename = f"{pathname}.conllu"
  CoNLL.write_doc2conll(doc, conllufilename)

In [5]:
tag_and_convert("expressen.txt")
tag_and_convert("Personer som är på spektrumet. Personer på spektrumet. Personer som har autism")

/content/Project9_data
/content/Project9_data


In [6]:
"""
!git remote -v
!git init
!git config --global user.email 'isali352@student.liu.se'
!git config --global user.name 'IKeeso'
!git add expressen.conllu
!git commit -m 'New commit'  # commit in Colab
!git push origin main
"""

"\n!git remote -v\n!git init\n!git config --global user.email 'isali352@student.liu.se'\n!git config --global user.name 'IKeeso'\n!git add expressen.conllu\n!git commit -m 'New commit'  # commit in Colab\n!git push origin main\n"

In [105]:
class SemanticAnalyzer:
  def __init__(self, file_path):
    self.file_path = file_path
    self.data = self.parse_conll_u(self.file_path)
    self.PFL_sentence_matches = self.person_first_lang(self.data)
    self.IFL_sentence_matches = self.identity_first_lang(self.data)

  def parse_conll_u(self, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
      data = {}
      sentence = []
      for line in file:
        line = line.strip()
        if line.startswith('# text'):
          sentence = []
        if line.startswith("# sent_id"):
          line = line.split("=")
          key = line[1].strip()
          sentence = []
        elif line == '':
          if sentence:
            data[key] = sentence
        else:
            sentence.append(line.split('\t'))
    return data

  def person_first_lang(self, data):
    PFL_sents = {}
    PFL_regx = re.compile(r"har|med", re.IGNORECASE)

    for id, sentence in data.items():
      if self.has_autism(data[id]):
        for token in sentence:
          if token[1] == "autism":
            if sentence[int(token[0])-2][1] == "och".lower():
              if PFL_regx.match(sentence[int(sentence[int(token[6])-1][int(6)])-1][1]):
                PFL_sents[id] = sentence
            if PFL_regx.match(sentence[int(token[0])-2][1]):
              PFL_sents[id] = sentence
    return PFL_sents

  def identity_first_lang(self, data):
    IFL_sents = {}
    IFL_regx = re.compile(r"^.*(autistisk|autistiska)", re.IGNORECASE)

    for id, sentence in data.items():
      words = ' '.join(token[1] for token in sentence)
      if IFL_regx.match(words):
        IFL_sents[id] = sentence

    return IFL_sents


  def has_autism(self, sentence):
    autism_regx = re.compile(r"^.*\bautism|spektrumet")
    words = ' '.join(token[1] for token in sentence)
    if autism_regx.match(words):
      return True
    else:
      return False

# Example usage:
analyzer = SemanticAnalyzer(file_path='expressen.conllu')
tester = SemanticAnalyzer(file_path='Personer som är på spektrumet.conllu')

def main(file, lang):
    analyzer = SemanticAnalyzer(file_path=f"{file.lower()}.conllu")
    if lang.lower() == "pfl":
        matches = analyzer.PFL_sentence_matches
    elif lang.lower() == "ifl":
        matches = analyzer.IFL_sentence_matches
    else:
        print("Invalid language choice. Please choose either 'PFL' or 'IFL'.")
        return

    for id, sentence in matches.items():
        print(f"Sentence ID: {id} | {' '.join(token[1] for token in sentence)}")
        print("")

    print(f"Total matching sentences: {len(matches)}")

if __name__ == "__main__":
    file_path = input("Enter the file path: ")
    lang = input("Enter the language (PFL or IFL): ")
    main(file_path, lang)

Enter the file path: expressen
Enter the language (PFL or IFL): pfl
Sentence ID: 4 | Många med autism känner sig missförstådda och har svårt att klara skolan .

Sentence ID: 6 | – Varannan elev med autism når inte målen i skolan , på högstadiet är andelen ännu större .

Sentence ID: 8 | Det är viktigt att öka förståelsen för att olika personer med autism har olika förutsättningar , menar Linn .

Sentence ID: 11 | Förändrade attityder krävs för att skapa ett samhälle för alla Sedan 2018 har Postkodlotteriet stöttat Autism Sverige , tidigare Autism- och Aspergerförbundet , som arbetar för att förbättra villkoren för personer med autism .

Sentence ID: 19 | – Tack vare alla lottköpare kan vi arbeta effektivare för att nå vår vision , ett samhälle där alla kan delta , där personer med autism möts med respekt och har bra livskvalitet genom hela livet .

Sentence ID: 25 | – Eftersom jag stöter på människor med autism i mitt jobb så är det väldigt spännande att höra vad Autism Sverige gör mer